# Tutorial 2b: Running a generic single-point case

This tutorial is an introduction to running single-point simulations of the Community Terrestrial Systems Model (CTSM).  It will guide you through setting up the required driver data (i.e. surface and atmosphere data) as well as setting up and submitting a single-point CTSM case.

In the previous tutorial, `Day1_GlobalCase`, we set up and ran a global CTSM case. Many of the steps required to run a single-point case are similar, with some changes and additional required steps which we will cover here.
<br><br>

### Questions about this tutorial? 
- Please post them on the [CTSM forum in the CESM Bulletin Board](https://bb.cgd.ucar.edu/cesm/forums/ctsm-clm-mosart-rtm.134/). Note that this resource will require you to register and log in so that you can be notified of responses to your inquiries. 
- You can also file issues on the [NCAR CTSM-Tutorial GitHub repository](https://github.com/NCAR/CTSM-Tutorial-2022).
***

## In this tutorial

The tutorial has several components. Below you will find steps to: 
1. Generate subset surface and atmosphere data files at a single latitude and longitude point.
2. Set up and submit a single-point case.
3. Basic visualization of single-point model history files.

## Note This tutorial assumes that you've completed the previous tutorials!

If you haven't downloaded CTSM from the github repository you need to go back to the `NEON_Simulation_Tutorial` and do this first. If you haven't completed the Day 1 tutorial (`Day1_GlobalCase`), go back and do this first.
***

<h1> 1. Subset global surface and atmosphere files </h1>

CTSM uses a surface data file to read in important grid cell-level information like vegetation, crop, and glacier grid cell fractions, the fractional cover of each plant functional type (PFT), and soil characteristics.

A global surface data file is located and read by default for global CTSM cases, depending on the chosen _component set_ and _resolution_. To run CTSM at a single point, we will need to supply a surface data file at a specified latitude and longitude.


<div class="alert alert-block alert-info">
<b>Note:</b> Remember from Day 1 that a **component set**, or coloquially a "compset", specifies a configuration for your case, including the component models, time period of simulation, and physics options. The **resolution** defines the model resolution or grid.
</div>


Similarly, when running in _DATM mode_, that using a "data atmosphere model", with climate data (e.g. temperature, precipitation, solar radiation, etc.) driven by an input file, CTSM needs DATM files. We can also provide subset global DATM for single-point runs.

<div class="alert alert-block alert-info">
<b>Note:</b> It is not strictly required to provide CTSM/CIME with subset DATM data in order to run a single-point case, as CTSM can just use the global files. However, your runs will be much faster if you use subset climate data.
</div>

<h2> 1.1 Use <em>subset_data</em> to subset surface and DATM files</h2>

We have created a python script, *subset_data*, which will subset default global surface and DATM files at a user-specified latitude and longitude.